# LAB GenAI - LLMs

# OpenAI GPT API Exercises

## 1. Basic Conversation
**Exercise:** Create a simple chatbot that can answer basic questions about a given topic (e.g., history, technology).  
**Parameters to explore:** `temperature`, `max_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `n`, `stop`.

Comment what happen when you change the parameters 
(read documentation!)

In [14]:
import openai

# load and set our key
openai_api_key = 'sk-proj-Ca6RLOnhFQTOevVv2tMGT3BlbkFJlwaoLQqxcdb2eDDxaORi'

In [15]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=openai_api_key,
)

In [20]:
def gpt_response(inp, message_history, **params):
    # We save the user's input
    message_history.append({"role": "user", "content": f"{inp}"})

    # Generate a response from the chatbot model
    completion_params = {
        "model": "gpt-3.5-turbo",
        "messages": message_history,
        **params  # Include additional parameters
    }

    chat_completion = client.chat.completions.create(**completion_params
        )

    # We save the assistant response
    message_history.append({"role": "assistant", "content": f"{chat_completion.choices[0].message.content}"})

    return message_history

message_history=[{"role": "system", "content": "Complete the prompt."}]

for i in [0,1,2]:
    message_history = gpt_response("My favourite animal is ",
                                   message_history,
                                   max_tokens=100,
                                   temperature=i)
    print(message_history[-1]["content"]+"\n") # Print the last response

the red panda.

the majestic elephant.

the playful dolphin.



In [21]:
message_history=[{"role": "system", "content": "Complete the prompt."}]

for i in [0,1]:
    message_history = gpt_response("My favourite animal is ",
                                   message_history,
                                   max_tokens=100,
                                   top_p=i)
    print(message_history[-1]["content"]+"\n") # Print the last response

a dolphin.

a red panda.



In [22]:
message_history=[]

for i in [-2,2]:
    message_history = gpt_response("generate 20 ways to say you can't buy that because you're broke",
                                   message_history,
                                   max_tokens=100,
                                   presence_penalty=i,
                                   frequency_penalty=i)
    print(message_history[-1]["content"]+"\n")# Print the last response

1. I can't afford it right now. 
2. I don't have the money for it. 
3. I'm a little short on cash. 
4. I'm on a tight budget. 
5. I'm low on funds. 
. 
. 
. 
. 
. 
. 
. 
. 
. 
. 
...........................

1. My wallet is feeling a little light at the moment.
2. I'm financially strapped right now.
3. Money's tight for me currently.
4. Unfortunately, my bank account is running low.
5. I've hit a rough patch with finances recently.
6.I can't swing that purchase due to financial constraints
7.I'm not in a position to make that buy because of money worries 
8.Im struggling financially so cant afford it sorry 
9.Sadly, funds are depleted



**Temperature (temperature):**

- Effect: Controls the randomness of the predictions. Higher values make the model more creative by sampling more diverse outputs. Lower values make it more deterministic.
Example: Setting temperature=0.5 might produce more predictable and conservative answers, while temperature=1.0 could generate more varied and surprising responses.

**Max Tokens (max_tokens):**

- Effect: Limits the maximum number of tokens (words or subwords) in the response. It determines the length of the generated text.
Example: If max_tokens=50, the model will generate responses that are up to 50 tokens long.

**Top-p (top_p):**

- Effect: A probabilistic sampling technique where the model considers only the most likely tokens whose cumulative probability mass is greater than top_p.
Example: top_p=0.9 means the model samples from tokens whose cumulative probability adds up to 90% until it chooses the next token.

**Frequency Penalty (frequency_penalty) and Presence Penalty (presence_penalty):**

- Effect: These penalties adjust the distribution of tokens in the response. Frequency penalty discourages repetitive phrases, while presence penalty encourages a diverse range of words.
Example: Increasing frequency_penalty (> 0) reduces repetitive responses, while increasing presence_penalty (> 0) promotes diversity in vocabulary.

**Nucleus Sampling (top_p vs top_k):**

- Effect: top_p (nucleus sampling) and top_k (top-k sampling) are mutually exclusive and control how the model selects the next word.
Example: top_p=0.9 (nucleus sampling) restricts the model to the most likely tokens that make up 90% of the probability mass, whereas top_k=50 (top-k sampling) limits consideration to the top 50 likely tokens.

**Stop (stop):**

Effect: Controls when the model should stop generating text. It can be either a specific token or a set of tokens that indicate the end of the response.
Example: Setting stop=[".", "?"] tells the model to stop generating once it encounters either a period or a question mark.

**How Parameters Affect Responses:**

**Temperature:** Higher values lead to more varied and sometimes more imaginative responses, while lower values produce more predictable and safer responses.

**Max Tokens:** Affects the length of the response. Larger values allow for more detailed answers, while smaller values constrain the response length.

**Top-p and Top-k:** Influence the diversity and specificity of the vocabulary used in responses. Top-p allows for more varied responses based on cumulative probabilities, while top-k limits choices to the most probable tokens.

**Frequency and Presence Penalties:** Impact the style and diversity of language in responses. Adjusting these penalties can help control the repetitiveness and creativity of the model's outputs.

**Stop Tokens:** Define when the model should end its response, which is useful for controlling the completeness of answers or preventing overly long outputs.

## 2. Summarization
**Exercise:** Write a script that takes a long text input and summarizes it into a few sentences.  
**Parameters to explore:** `temperature`, `max_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `best_of`, `logprobs`.

Comment what happen when you change the parameters 
(read documentation!)

## 3. Translation
**Exercise:** Develop a tool that translates text from one language to another using the API.  
**Parameters to explore:** `temperature`, `max_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `echo`, `logit_bias`.

Comment what happen when you change the parameters 
(read documentation!)

In [25]:
from datasets import load_dataset
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

huggingface_dataset_name = "knkarthick/dialogsum"
# Check if you need to specify additional parameters like `name`
dataset = load_dataset(huggingface_dataset_name)

model_name='google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
        
        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""
    
    dialogue = dataset['test'][example_index_to_summarize]['dialogue']
    
    prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""
        
    return prompt



Found cached dataset csv (C:/Users/Francesco Corda/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [30]:
import openai

# Set your OpenAI API key
api_key = 'sk-proj-Ca6RLOnhFQTOevVv2tMGT3BlbkFJlwaoLQqxcdb2eDDxaORi'
openai.api_key = api_key


def summarize_text(text, temperature=0.7, max_tokens=150, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, best_of=1, logprobs=None):
  """
  Summarizes a long text input using OpenAI GPT-3 API.

  Args:
      text: The long text to summarize.
      temperature: Controls the randomness of the generated text (higher = more random).
      max_tokens: The maximum number of tokens to generate in the summary.
      top_p: Influences which words are more likely to be chosen (higher = more common words).
      frequency_penalty: Penalty for words that appear often in the input text.
      presence_penalty: Penalty for words that appear multiple times within the generated text.
      best_of: Number of independent generations to try for getting the best summary.
      logprobs: Whether to return the log probabilities of the generated text.

  Returns:
      A string containing the summarized text.
  """
  prompt = f"Summarize the following text in a few sentences:\n{text}"
  engine = "text-davinci-003"  # Choose a suitable GPT-3 engine for summarization

  response = None
  for _ in range(best_of):
    response = openai.Completion.create(
        engine=engine,
        prompt=prompt,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        logprobs=logprobs
    )
  
  summary = response.choices[0].text.strip()
  return summary

# Example usage
long_text = """This is a long text that needs to be summarized. It contains multiple paragraphs 
with important information spread throughout. The goal is to condense this text into a 
shorter version that captures the key points. By using summarization techniques, we can 
extract the essence of the text and improve readability. This can be particularly useful 
for long articles, research papers, or any content that requires a quick understanding 
of the main ideas."""

summary = summarize_text(long_text)
print(summary)



APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


## 4. Sentiment Analysis
**Exercise:** Implement a sentiment analysis tool that determines the sentiment of a given text (positive, negative, neutral).  
**Parameters to explore:** `temperature`, `max_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `n`, `logprobs`.

Comment what happen when you change the parameters 
(read documentation!)

In [31]:
# Lets give an example of sentiment analysis
message_history=[{"role": "system", "content": "Decide whether a Tweet's sentiment is positive, neutral, or negative."},
             {"role": "user", "content": "Tweet: \"I loved the new Batman movie!\"\nSentiment:"},
              {"role": "assistant", "content": "Positive"}
  ]
print("Write a tweet: ")

while(True):
    message_history = gpt_response(input("> "),
                                   message_history,
                                   temperature=0,
                                   max_tokens=60,
                                   frequency_penalty=0.5)
    print(message_history[-1]["content"])

    

Write a tweet: 
Positive
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral


KeyboardInterrupt: Interrupted by user

## 5. Text Completion
**Exercise:** Create a text completion application that generates text based on an initial prompt.  
**Parameters to explore:** `temperature`, `max_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `stop`, `best_of`.

Comment what happen when you change the parameters 
(read documentation!)

In [16]:
messages=[ # messages parameter must be a list of dictionaries
    # can be as short as one message or many back and forth turns.
    {"role": "system", "content": "You are a famous chef. Share your best cooking tips and tricks."}, # each dictionary has a role and content
    {"role": "user", "content": "What are the ingredients for making pancakes?"},
  ]

chat_completion = client.chat.completions.create(
    messages = messages,
    model = "gpt-3.5-turbo",
)

answer = chat_completion.choices[0].message.content
print(answer)

To make classic pancakes, you'll need the following ingredients:

- 1 cup all-purpose flour
- 2 tablespoons sugar
- 1 tablespoon baking powder
- 1/2 teaspoon salt
- 1 cup milk
- 1 large egg
- 2 tablespoons melted butter or oil
- Additional butter or oil for greasing the pan

Optional ingredients for flavor variations:
- Vanilla extract
- Cinnamon
- Blueberries, chocolate chips, or diced fruits

Remember to mix the wet and dry ingredients separately before combining them to ensure a smooth batter. Cooking pancakes on medium-low heat will help prevent burning and ensure they cook evenly.


# BONUS: Google Vertex AI

## 1. Basic Conversation
**Exercise:** Create a basic chatbot using Google Vertex AI to answer questions about a given topic.  
**Parameters to explore:** `temperature`, `max_output_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `n`, `stop`.

Comment what happen when you change the parameters 
(read documentation!)

## 2. Summarization
**Exercise:** Develop a script that summarizes long text inputs using Google Vertex AI.  
**Parameters to explore:** `temperature`, `max_output_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `best_of`, `logprobs`.

Comment what happen when you change the parameters 
(read documentation!)

## 3. Translation
**Exercise:** Create a tool that translates text from one language to another using Google Vertex AI.  
**Parameters to explore:** `temperature`, `max_output_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `echo`, `logit_bias`.

Comment what happen when you change the parameters 
(read documentation!)

## 4. Sentiment Analysis
**Exercise:** Implement a sentiment analysis tool using Google Vertex AI to determine the sentiment of a given text.  
**Parameters to explore:** `temperature`, `max_output_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `n`, `logprobs`.

Comment what happen when you change the parameters 
(read documentation!)

## 5. Text Completion
**Exercise:** Develop a text completion application using Google Vertex AI to generate text based on an initial prompt.  
**Parameters to explore:** `temperature`, `max_output_tokens`, `top_p`, `frequency_penalty`, `presence_penalty`, `stop`, `best_of`.

Comment what happen when you change the parameters 
(read documentation!)